# Build and train a sample autoencoder model

In [10]:
from platform import python_version

print("Python Version: ",python_version())

import tensorflow as tf
print("TensorFlow Version: ",tf.version.VERSION)

Python Version:  3.6.13
TensorFlow Version:  2.1.3


## Model definition

In [13]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers

#[5,3,2,3,5]


# Layer 1 input size
input_dim = 5
encoding_dim = 3

#Activation Function
afn="tanh"

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation=afn,activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim), activation=afn)(encoder)
encoder = Dense(int(2), activation=afn)(encoder) # bottleneck
decoder = Dense(int(encoding_dim), activation=afn)(encoder)
decoder = Dense(int(encoding_dim), activation=afn)(decoder)
decoder = Dense(input_dim, activation=afn)(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 20  

## Training data

In [14]:
import numpy as np

X_train_scaled = np.random.rand(100, 5)
X_train_scaled.shape

(100, 5)

## Train model

In [15]:
import datetime

nb_epoch = 1
batch_size = 5
autoencoder.compile(optimizer='adam', loss='mse' )

t_ini = datetime.datetime.now()

# X_train_scaled, X_train_scaled (Unsupervised Learning)

history = autoencoder.fit(x=X_train_scaled, y=X_train_scaled,
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_split=0.1,
                        verbose=1
                        ).history

t_fin = datetime.datetime.now()
print('Time to run the model: {} Sec.'.format((t_fin - t_ini).total_seconds()))

Train on 90 samples, validate on 10 samples
90/90 [==============================] - 1s 8ms/sample - loss: 0.3896 - val_loss: 0.3334
Time to run the model: 0.739431 Sec.


## Save model to SavedModel format which is compatible with Tensorflow Serving

In [16]:
!mkdir -p model_sherry
autoencoder.save('model_sherry/1/', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_sherry/1/assets


In [17]:
!saved_model_cli show --all --dir 'model_sherry/1/'


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: serving_default_input_3:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_11'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Option

## Create a model archive file (model.tar.gz) and upload to s3

In [19]:
!tar -C "$PWD" -czf model.tar.gz model_sherry/

In [20]:
from sagemaker.session import Session

model_data = Session().upload_data(path='model.tar.gz', key_prefix='ised_demo_model')
print('model uploaded to: {}'.format(model_data))

model uploaded to: s3://sagemaker-us-east-2-240487350066/ised_demo_model/model.tar.gz


## Wrap into a SageMaker TensorflowModel object and deploy to a real-time endpoint

In [21]:
from sagemaker.tensorflow import TensorFlowModel
from sagemaker import get_execution_role
# from sagemaker.deserializers import JSONDeserializer
# from sagemaker.serializers import CSVSerializer

sagemaker_role = get_execution_role()

model = TensorFlowModel(model_data=model_data, role=sagemaker_role, framework_version='2.1.3')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge', serializer=CSVSerializer(), deserializer=JSONDeserializer())

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

## Make predictions

In [22]:
# The gamma model expects an input of shape [1,5]
gamma_input = np.array([[0.03436834, 0.05735075, 0.03385664, 0.23529412, 0.0223518 ]])

#model_input = gamma_input.reshape(1, 5)

print(gamma_input.shape)
gamma_input

(1, 5)


array([[0.03436834, 0.05735075, 0.03385664, 0.23529412, 0.0223518 ]])

In [23]:
predictor.predict(gamma_input)

{'predictions': [[0.171888843,
   0.0929558352,
   0.100073166,
   -0.0468236022,
   -0.0452863686]]}

In [28]:
# delete endpoint
predictor.delete_endpoint()

### When using a lambda function to invoke the endpoint, the code would be something like:

In [ ]:
# import boto3

# endpoint_name = predictor.endpoint_name
# runtime_client = boto3.client('runtime.sagemaker')
# response = runtime_client.invoke_endpoint(EndpointName = endpoint_name, 
#                                    ContentType = 'specify_content_type', 
#                                    Body = your_payload)